# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.4. Instalar librerías necesarias
(descomenta si es la primera vez que corres el codigo)

In [17]:
# %pip install gym==0.17.3
# %pip install git+https://github.com/Kojoley/atari-py.git
# %pip install pyglet==1.5.0
# %pip install h5py==3.1.0
# %pip install Pillow==9.5.0
# %pip install keras-rl2==1.0.5
# %pip install Keras==2.2.4
# %pip install tensorflow==2.5.3
# %pip install torch==2.0.1
# %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [18]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [19]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
IN_COLAB=False

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [20]:
print("Numero de acciones disponibles:" + str(nb_actions))
print("Formato de las observaciones:")
env.observation_space

Numero de acciones disponibles:6
Formato de las observaciones:


Box(0, 255, (210, 160, 3), uint8)

Un espacio de soluciones de dimensiones (210,160,3) tiene un tamaño inmanejable. Se puede reducir ese espacio a un tamaño de 84x84 con una window length 4.

In [21]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 1. Implementación de la red neuronal

Se puede hacer:


1.   DQN
2.   Doble DQN
3.   Duel DQN

Y luego también variar el discount factor, cambios en epsilon, tamaño de memoria, uso de epsilon greedy vs. boltzman y escribir como varía el resultado. Cambiar el número de episodios es un poco gilipollez, dara mejores resultados por más train. Usamos un número pequeño de episodios. el mínimo para que llegue a 20 puntos y a partir de ahí vamos probando. También podríamos cambiar el input de la red neuronal.

## VER EN CONJUNTO





In [22]:
# arquitectura de DeepMind paper
def construct_double_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.000001)):

        input_layer = Input(shape=input_shape)
        permuted_layer = Permute((3, 2, 1))(input_layer)   #IMPORTANTE 
        conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(permuted_layer)
        conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
        conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
        flatten = Flatten()(conv3)

        fc1 = Dense(512)(flatten)
        advantage = Dense(num_actions)(fc1)

        fc2 = Dense(512)(flatten)
        value = Dense(1)(fc2)

        policy = concatenate([advantage, value], axis=-1)
        #policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)
        def custom_lambda(x):
            adv = x[:, :-1]
            val = x[:, -1:]
            print(f"adv shape: {adv.shape}, val shape: {val.shape}")
            return adv - K.mean(adv, axis=1, keepdims=True) + val

        policy = Lambda(custom_lambda)(policy)


        model = Model(inputs=input_layer, outputs=policy)
        model.compile(loss=loss, optimizer=optimizer)


        print("Successfully constructed networks.")

        return model

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Ver aquí parámetros del https://www.tensorflow.org/agents/api_docs/python/tf_agents/agents/DqnAgent

In [23]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = EpsGreedyQPolicy()

In [24]:
model = construct_double_q_network(input_shape=(4, 84,84), num_actions=6, loss='mse', optimizer=Adam(lr=0.000001))
model.summary()


adv shape: (None, 6), val shape: (None, 1)
Successfully constructed networks.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4, 84, 84)]  0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 84, 84, 4)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 20, 20, 32)   8224        permute_1[0][0]                  
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 9, 9, 64)     32832       conv2d_3[0][0]                   
______________

In [25]:
# dqn = DQNAgent(model=model, 
#                 nb_actions=nb_actions, 
#                 policy=policy,
#                memory=memory, 
#                processor=processor,
#                nb_steps_warmup=50000, 
#                gamma=.99,
#                target_model_update=10000,
#                train_interval=4)

dqn = DQNAgent(model=model, 
                nb_actions=nb_actions,
                processor=processor,
                memory=memory,
                nb_steps_warmup=50000,
                target_model_update=1e-2, 
                policy=policy) #target model te congela durante 100 steps lo que se usa para el "ground truth"

dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

adv shape: (None, 6), val shape: (None, 1)


In [112]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1750000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: 0.0087
11 episodes - episode_reward: 7.909 [1.000, 15.000] - ale.lives: 2.069

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 82s 8ms/step - reward: 0.0095
13 episodes - episode_reward: 6.923 [1.000, 14.000] - ale.lives: 2.087

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 82s 8ms/step - reward: 0.0125
13 episodes - episode_reward: 9.692 [0.000, 19.000] - ale.lives: 2.098

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 85s 9ms/step - reward: 0.0089
13 episodes - episode_reward: 7.077 [2.000, 13.000] - ale.lives: 2.037

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 0.0098
13 episodes - episode_reward: 7.077 [2.000, 15.000] - ale.lives: 2.091

Interval 6 (50000 steps performed)
10000/10000 [=====

In [26]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights_250000.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...


c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 17.000, steps: 930
Episode 2: reward: 5.000, steps: 391
Episode 3: reward: 8.000, steps: 554
Episode 4: reward: 6.000, steps: 483
Episode 5: reward: 25.000, steps: 1061
Episode 6: reward: 20.000, steps: 1035
Episode 7: reward: 21.000, steps: 967
Episode 8: reward: 10.000, steps: 654
Episode 9: reward: 14.000, steps: 807
Episode 10: reward: 19.000, steps: 978


In [27]:

#checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
#callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks = [FileLogger(log_filename, interval=100)]


dqn.fit(env, callbacks=callbacks, nb_steps=250000, log_interval=10000, visualize=False)

weights_filename = 'dqn_{}_weights_500000.h5f'.format(env_name)
dqn.save_weights(weights_filename, overwrite=True)

Training for 250000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 1:44 - reward: 0.0000e+00

10000/10000 [==============================] - 69s 7ms/step - reward: 0.0178
14 episodes - episode_reward: 12.714 [3.000, 20.000] - ale.lives: 2.109

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: 0.0184
13 episodes - episode_reward: 12.692 [1.000, 20.000] - ale.lives: 2.055

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 68s 7ms/step - reward: 0.0177
14 episodes - episode_reward: 13.714 [4.000, 25.000] - ale.lives: 1.945

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: 0.0194
11 episodes - episode_reward: 17.091 [8.000, 24.000] - ale.lives: 2.035

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 68s 7ms/step - reward: 0.0174
13 episodes - episode_reward: 13.692 [4.000, 30.000] - ale.lives: 1.947

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 1512s 151ms/step - reward: 0

In [ ]:
weights_filename = 'dqn_{}_weights_500000.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---